In [1]:
from solana.rpc.api import Client
from solders.pubkey import Pubkey
from solders.rpc.responses import GetAccountInfoResp
from solders.account import Account
import requests

token_address = "CniPCE4b3s8gSUPhUiyMjXnytrEqUrMfSsnbBjLCpump" # this will be fed from a previous step (CA)

def find_pools_dexscreener(token_address: str):
    url = f"https://api.dexscreener.com/latest/dex/tokens/{token_address}"
    response = requests.get(url)
    return response.json().get("pairs", [])

# Call the function to get the pairs
pairs = find_pools_dexscreener(token_address)

# Initialize the variable for the Raydium pool address
raydium_pool = None

# Loop through the pairs to find the Raydium pool
for pair in pairs:
    if pair.get('dexId') == 'raydium':
        raydium_pool = pair.get('pairAddress')
        break  # Exit the loop once the Raydium pool is found

# print(pairs)
find_pools_dexscreener(token_address)
# print('raydium pool address:', raydium_pool)

[{'chainId': 'solana',
  'dexId': 'raydium',
  'url': 'https://dexscreener.com/solana/9fmdkqipjk2teeuv53bmdxi52urlbrevv38k8gbnkim7',
  'pairAddress': '9fmdkQipJK2teeUv53BMDXi52uRLbrEvV38K8GBNkiM7',
  'baseToken': {'address': 'CniPCE4b3s8gSUPhUiyMjXnytrEqUrMfSsnbBjLCpump',
   'name': 'PWEASE',
   'symbol': 'pwease'},
  'quoteToken': {'address': 'So11111111111111111111111111111111111111112',
   'name': 'Wrapped SOL',
   'symbol': 'SOL'},
  'priceNative': '0.0002695',
  'priceUsd': '0.03410',
  'txns': {'m5': {'buys': 37, 'sells': 24},
   'h1': {'buys': 649, 'sells': 777},
   'h6': {'buys': 3094, 'sells': 2903},
   'h24': {'buys': 28424, 'sells': 25456}},
  'volume': {'h24': 17308059.12,
   'h6': 2037435.53,
   'h1': 644499,
   'm5': 20127.08},
  'priceChange': {'m5': 0.5, 'h1': 13.79, 'h6': 26.51, 'h24': 63.88},
  'liquidity': {'usd': 1336581.73, 'base': 19617169, 'quote': 5273.8684},
  'fdv': 34108945,
  'marketCap': 34108945,
  'pairCreatedAt': 1740952868000,
  'info': {'imageUrl': 'ht

Time Conversion 

In [13]:
# Convert to human-readable time
import datetime
timestamp = 1740952868000 / 1000  # Convert to seconds
print(datetime.datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S'))

2025-03-02 22:01:08


Earliest Raydium LP

In [19]:
from solana.rpc.api import Client
from solders.pubkey import Pubkey
from solders.rpc.responses import GetAccountInfoResp
from solders.account import Account
import requests
from datetime import datetime

token_address = "7ryukpbkAdDaMgHN8TroVVuTmKpNh796dNU8VXKUpump" # this will be fed from a previous step (CA)

def find_pools_dexscreener(token_address: str):
    url = f"https://api.dexscreener.com/latest/dex/tokens/{token_address}"
    response = requests.get(url)
    return response.json().get("pairs", [])

def get_earliest_raydium_pool(pairs):
    """
    Finds the Raydium pool with the earliest creation timestamp
    
    Args:
        pairs (list): List of trading pairs from DexScreener
        
    Returns:
        tuple: (pairAddress, creation_date) of the earliest Raydium pool, or (None, None) if none found
    """
    raydium_pairs = [pair for pair in pairs if pair.get('dexId') == 'raydium']
    
    if not raydium_pairs:
        return None, None
    
    # Find the pair with the minimum timestamp
    earliest_pair = min(raydium_pairs, key=lambda x: x.get('pairCreatedAt', float('inf')))
    
    # Convert timestamp to human-readable date
    timestamp = earliest_pair.get('pairCreatedAt')
    if timestamp:
        creation_date = datetime.fromtimestamp(timestamp / 1000)
    else:
        creation_date = None
    
    return earliest_pair.get('pairAddress'), creation_date

# Call the function to get the pairs
pairs = find_pools_dexscreener(token_address)

# Get the earliest Raydium pool
raydium_pool_address, creation_date = get_earliest_raydium_pool(pairs)

print(f"Earliest Raydium pool: {raydium_pool_address}")
if creation_date:
    print(f"Created on: {creation_date}")

Earliest Raydium pool: B6DTnnru2Vz7mTW2bPFJFs22dVCDeFKAPufrRFcFB7Co
Created on: 2025-03-10 07:58:45


Getting Solana Data from a LP pair

In [6]:
import base64
from solana.rpc.api import Client
from solders.pubkey import Pubkey

RPC_URL = "https://api.mainnet-beta.solana.com"
client = Client(RPC_URL)

account_address = Pubkey.from_string("DTPUUPRxnHAdM3DoiDiUf7M3mym8GXdp6A97AJEZpump")
resp = client.get_account_info(account_address)

account_info = resp.value
if account_info:
    # Basic account information
    print("=== Account Information ===")
    print(f"Address: {account_address}")
    print(f"Owner Program: {account_info.owner}")
    print(f"Executable: {account_info.executable}")
    print(f"Rent Epoch: {account_info.rent_epoch}")
    
    # Balance information
    sol_balance = account_info.lamports / 1_000_000_000
    print(f"\n=== Balance ===")
    print(f"SOL Balance: {sol_balance:.9f} SOL")
    print(f"Lamports: {account_info.lamports}")
    
    # Data information
    print(f"\n=== Data ===")
    print(f"Data Length: {len(account_info.data)} bytes")
    print(f"Raw Data (first 64 bytes): {account_info.data[:64].hex()}") #
else:
    print("Account not found")

=== Account Information ===
Address: DTPUUPRxnHAdM3DoiDiUf7M3mym8GXdp6A97AJEZpump
Owner Program: TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA
Executable: False
Rent Epoch: 18446744073709551615

=== Balance ===
SOL Balance: 9.788665040 SOL
Lamports: 9788665040

=== Data ===
Data Length: 82 bytes
Raw Data (first 64 bytes): 0000000006c5c1ce638d2567d26468b05eb951d1a28dcc6e123482b5c675149770e62bf2d2683d23758d03000601000000000000000000000000000000000000
